<img src="Pictures/RedfishLogo.png" alt="Redfish Logo" style="width: 250px;"/> 

Version 0.36

# OData queries

## Introduction

This Jupyter notebook demonstrates several OData queris (`$expand`, `only`...) using [Bash](https://www.gnu.org/software/bash/) and the [cURL](https://curl.haxx.se/) tool against an HPE iLO 5. For didactic reasons, commands presented in this notebook may be not optimized and don't follow the recommended [best practises](https://developer.hpe.com/blog/getting-started-with-the-redfish-api-part-2).

**[OData](https://www.odata.org/)** is an open protocol to allow the creation and consumption of queryable and interoperable RESTful APIs in a simple and standard way.

More details are in the [HPE Redfish API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#introduction).



### `$expand` query parameter

![ExpandSupport](Pictures/ExpandSupport.png)

### `only` Query parameter

![OnlyQueryParamter](Pictures/OnlyQueryParameter.png)






## Create environment variables and session

The following `bash` code defines environment variables (i.e. IP address, username, password....) depending on your student ID number stored in variable `$Stud`. It creates as well several `.json` files containing various HTTP workloads required to POST or PATCH the managed iLO. 

In [1]:
# Create BMC related variables
iLO5_IP=ilo-etc1
#iLO5_IP=172.16.50.99
iLO5_URI="https://${iLO5_IP}"

# iLO 5 Administrator credentials
iLO5_User="student"
iLO5_Passwd='P@ssw0rd!'

# Minimum required Redfish headers
HeaderODataVersion="OData-Version: 4.0"
HeaderContentType="Content-Type: application/json"

# Data files
ResponseHeaders="ResponseHeaders.txt"             # Used to hold HTTP response headers
SessionData="./CreateSession-data.json"           # Body/Workload used to create the Redfish session

cat > ${SessionData} << __EOF__
{
        "UserName": "$iLO5_User",
        "Password": "$iLO5_Passwd"
}
__EOF__


# Verify we can reach the remote Bmc
curl --insecure --silent \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --request GET ${iLO5_URI}/redfish | jq  || echo "WARNING: Problem reaching the remote BMC"
  

{
  "v1": "/redfish/v1/"
}


## Create the Redfish session

Redfish allows basic authentication and session authentication. With basic authentication you need to supply the required credentials at each and every HTTP request. Session oriented authentication is achieved by requesting a `Token` that will be sent in the headers of all requests until the removal of the session.

To get this `Token`, POST a session request with the remote BMC credentials in its body. The `Token` as well as the `session location` will be in the headers of the response.

In [2]:
echo 'Create iLO 5 Session' 

curl --dump-header  $ResponseHeaders \
     --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --request POST --data "@$SessionData" \
     ${iLO5_URI}/redfish/v1/SessionService/Sessions > /dev/null 2>&1
     
Token=$(awk '/X-Auth-Token/ {print $NF}' $ResponseHeaders | tr -d '\r')
SessionLocation="$iLO5_URI"$(awk '/^Loca.*Se/ {gsub("https://.*/red", "/red", $NF);print $NF}' $ResponseHeaders | tr -d '\r')

echo "Token: $Token"
echo -e "Session Location: $SessionLocation\n"

Create iLO 5 Session
Token: d9458dc203cd439660291c3173860e04
Session Location: https://ilo-etc1/redfish/v1/SessionService/Sessions/student000000005f2277edef16a436



### Retrieve iLO 5 supported OData protocol features

The following command retrieves the OData supported protocol features.

In [3]:
echo "Retrieve supported OData features:"
curl --insecure --silent   \
     --header  "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}/redfish/v1 | jq '.ProtocolFeaturesSupported'


Retrieve supported OData features:
{
  "ExpandQuery": {
    "ExpandAll": false,
    "Levels": true,
    "Links": false,
    "MaxLevels": 1,
    "NoLinks": true
  },
  "FilterQuery": true,
  "SelectQuery": false
}


###  GET collection without `$expand` 


In [4]:
echo "GET FirmwareInventory collection"

curl --insecure  --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}/redfish/v1/UpdateService/FirmwareInventory | jq 
       

GET FirmwareInventory collection
{
  "@odata.context": "/redfish/v1/$metadata#SoftwareInventoryCollection.SoftwareInventoryCollection",
  "@odata.etag": "W/\"D4A4F174\"",
  "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory",
  "@odata.type": "#SoftwareInventoryCollection.SoftwareInventoryCollection",
  "Description": "Firmware Inventory Collection",
  "Name": "Firmware Inventory Collection",
  "Members": [
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/1"
    },
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/2"
    },
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/3"
    },
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/4"
    },
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/5"
    },
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/6"
    },
    {
      "@odata.id": "/redfish/v1/UpdateService/FirmwareInventory/7"
    },
    {
      "@odata.id

###  Expand collection with `$expand` 


In [5]:
echo "Retrieve ALL Firmware versions with a single GET"

curl --insecure  --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}'/redfish/v1/UpdateService/FirmwareInventory?$expand=.' | \
       jq '.Members | .[] | {Name: .Name, Description: .Description, Version: .Version}'

Retrieve ALL Firmware versions with a single GET
{
  "Name": "iLO 5",
  "Description": "SystemBMC",
  "Version": "1.43 May 23 2019"
}
{
  "Name": "System ROM",
  "Description": "SystemRomActive",
  "Version": "U32 v2.10 (05/21/2019)"
}
{
  "Name": "Intelligent Platform Abstraction Data",
  "Description": "PlatformDefinitionTable",
  "Version": "9.5.0 Build 15"
}
{
  "Name": "System Programmable Logic Device",
  "Description": "SystemProgrammableLogicDevice",
  "Version": "0x2E"
}
{
  "Name": "Power Management Controller Firmware",
  "Description": "PowerManagementController",
  "Version": "1.0.4"
}
{
  "Name": "Power Supply Firmware",
  "Description": "PowerSupplies",
  "Version": "1.00"
}
{
  "Name": "Power Supply Firmware",
  "Description": "PowerSupplies",
  "Version": "1.00"
}
{
  "Name": "Innovation Engine (IE) Firmware",
  "Description": "InnovationEngineFirmware",
  "Version": "0.2.1.2"
}
{
  "Name": "Server Platform Services (SPS) Firmware",
  "Description": "SPSFirmwareVersion

## Expand collection only if it contains one element

### Without the `only` query parameter

In [6]:
echo "Chassis collection"

curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}'/redfish/v1/Chassis' | jq

Chassis collection
{
  "@odata.context": "/redfish/v1/$metadata#ChassisCollection.ChassisCollection",
  "@odata.etag": "W/\"AA6D42B0\"",
  "@odata.id": "/redfish/v1/Chassis",
  "@odata.type": "#ChassisCollection.ChassisCollection",
  "Description": "Computer System Chassis View",
  "Name": "Computer System Chassis",
  "Members": [
    {
      "@odata.id": "/redfish/v1/Chassis/1"
    }
  ],
  "Members@odata.count": 1
}


### With the `only` query parameter


In [7]:
echo "Chassis collection"

curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}'/redfish/v1/Chassis?only' | jq

Chassis collection
{
  "@odata.context": "/redfish/v1/$metadata#Chassis.Chassis",
  "@odata.etag": "W/\"2BCAB4E6\"",
  "@odata.id": "/redfish/v1/Chassis/1",
  "@odata.type": "#Chassis.v1_6_0.Chassis",
  "Id": "1",
  "ChassisType": "RackMount",
  "Links": {
    "ManagedBy": [
      {
        "@odata.id": "/redfish/v1/Managers/1"
      }
    ],
    "ComputerSystems": [
      {
        "@odata.id": "/redfish/v1/Systems/1"
      }
    ]
  },
  "Manufacturer": "HPE",
  "Model": "ProLiant DL360 Gen10",
  "Name": "Computer System Chassis",
  "NetworkAdapters": {
    "@odata.id": "/redfish/v1/Chassis/1/NetworkAdapters"
  },
  "Oem": {
    "Hpe": {
      "@odata.context": "/redfish/v1/$metadata#HpeServerChassis.HpeServerChassis",
      "@odata.type": "#HpeServerChassis.v2_2_1.HpeServerChassis",
      "Actions": {
        "#HpeServerChassis.DisableMCTPOnServer": {
          "target": "/redfish/v1/Chassis/1/Actions/Oem/Hpe/HpeServerChassis.DisableMCTPOnServer"
        },
        "#HpeServerChassi

## More OData queries

In addition to the `$expand` and `only` OData queries Redfish implementations can support others like `$filter`, `$top` and `$skip`. 

### `$filter` query

Extract IML logs created after a specific date.

In [8]:
Filter="filter=Created+gt+'2020-04-01T08:27:03Z'"

curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}/redfish/v1/Systems/1/LogServices/IML/Entries'?$'$Filter | jq

{
  "@odata.context": "/redfish/v1/$metadata#LogEntryCollection.LogEntryCollection",
  "@odata.etag": "W/\"C143DD57\"",
  "@odata.id": "/redfish/v1/Systems/1/LogServices/IML/Entries",
  "@odata.type": "#LogEntryCollection.LogEntryCollection",
  "Description": "Integrated Management Logs view",
  "Name": "Integrated Management Logs",
  "Members": [
    {
      "@odata.context": "/redfish/v1/$metadata#LogEntry.LogEntry",
      "@odata.id": "/redfish/v1/Systems/1/LogServices/IML/Entries/78",
      "@odata.type": "#LogEntry.v1_0_0.LogEntry",
      "Id": "78",
      "Created": "2020-04-30T08:29:41Z",
      "EntryType": "Oem",
      "Message": "HPE Ethernet 1Gb 4-port 331i Adapter - NIC Connectivity status changed to OK for adapter in slot 0, port 1",
      "Name": "Integrated Management Log",
      "Oem": {
        "Hpe": {
          "@odata.context": "/redfish/v1/$metadata#HpeLogEntry.HpeLogEntry",
          "@odata.type": "#HpeLogEntry.v2_1_0.HpeLogEntry",
          "Categories": [
      

### `$top` query 

Extract the oldest Log records within all possible iLO log files.

In [9]:
# 
ODataQuery='?$top=1'

# Retrieve LogURIs locations
LogURIs=$(curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}/redfish/v1/Systems/1/LogServices | jq -r '.Members | .[] | ."@odata.id"')

# For each LogURI in LogURIs, retrieve the location of entries.
LogEntriesURIs=""
  for LogURI in $LogURIs ; do
    LogEntriesURI=$(curl --insecure --silent \
         --header "$HeaderContentType" --header "$HeaderODataVersion" \
         --header "X-Auth-Token: $Token" \
         --request GET ${iLO5_URI}$LogURI | jq -r '.Entries | ."@odata.id"')

    LogEntriesURIs="${LogEntriesURI} $LogEntriesURIs"
done

# Remove trailing space if any
LogEntriesURIs=$(echo ${LogEntriesURIs%%*()})
echo -e "Oldest Log Entries URIs: ${iLO5_URI}${LogEntriesURIs}${ODataQuery}\n"


curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}${LogEntriesURIs}${ODataQuery} | jq


Oldest Log Entries URIs: https://ilo-etc1/redfish/v1/Systems/1/LogServices/IML/Entries?$top=1

{
  "@odata.context": "/redfish/v1/$metadata#LogEntryCollection.LogEntryCollection",
  "@odata.etag": "W/\"7F85F5FA\"",
  "@odata.id": "/redfish/v1/Systems/1/LogServices/IML/Entries",
  "@odata.type": "#LogEntryCollection.LogEntryCollection",
  "Description": "Integrated Management Logs view",
  "Name": "Integrated Management Logs",
  "Members": [
    {
      "@odata.context": "/redfish/v1/$metadata#LogEntry.LogEntry",
      "@odata.id": "/redfish/v1/Systems/1/LogServices/IML/Entries/1",
      "@odata.type": "#LogEntry.v1_0_0.LogEntry",
      "Id": "1",
      "Created": "2019-08-22T03:06:32Z",
      "EntryType": "Oem",
      "Message": "IML Cleared ( user: System Administrator)",
      "Name": "Integrated Management Log",
      "Oem": {
        "Hpe": {
          "@odata.context": "/redfish/v1/$metadata#HpeLogEntry.HpeLogEntry",
          "@odata.type": "#HpeLogEntry.v2_1_0.HpeLogEntry",
    

### `$skip` query

Skip the first 75 log entries and print the rest.

In [10]:

curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}'/redfish/v1/Systems/1/LogServices/IML/Entries?$skip=75' | jq

{
  "@odata.context": "/redfish/v1/$metadata#LogEntryCollection.LogEntryCollection",
  "@odata.etag": "W/\"FAB93158\"",
  "@odata.id": "/redfish/v1/Systems/1/LogServices/IML/Entries",
  "@odata.type": "#LogEntryCollection.LogEntryCollection",
  "Description": "Integrated Management Logs view",
  "Name": "Integrated Management Logs",
  "Members": [
    {
      "@odata.context": "/redfish/v1/$metadata#LogEntry.LogEntry",
      "@odata.id": "/redfish/v1/Systems/1/LogServices/IML/Entries/76",
      "@odata.type": "#LogEntry.v1_0_0.LogEntry",
      "Id": "76",
      "Created": "2020-03-25T17:45:59Z",
      "EntryType": "Oem",
      "Message": "HPE Ethernet 1Gb 4-port 331i Adapter - NIC Connectivity status changed to OK for adapter in slot 0, port 1",
      "Name": "Integrated Management Log",
      "Oem": {
        "Hpe": {
          "@odata.context": "/redfish/v1/$metadata#HpeLogEntry.HpeLogEntry",
          "@odata.type": "#HpeLogEntry.v2_1_0.HpeLogEntry",
          "Categories": [
      

## Delete sessions

It is extremely important to delete Redfish sessions to avoid reaching the maximum number of opened sessions in a BMC, preventing any access to it. Read this [article](https://developer.hpe.com/blog/managing-ilo-sessions-with-redfish) for more detail.

In [11]:
echo "Body response of a session deletion:"

curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request DELETE $SessionLocation | jq

Body response of a session deletion:
{
  "error": {
    "code": "iLO.0.10.ExtendedInfo",
    "message": "See @Message.ExtendedInfo for more information.",
    "@Message.ExtendedInfo": [
      {
        "MessageId": "Base.1.0.Success"
      }
    ]
  }
}


## Wrap up

In this notebook you performed the following actions:

  * Create and delete a token based Redfish session
  * Retrieved iLO 5 supported OData protocol features
  * Expand a collection with OData `$expand` query operator
  * Expand a collection with OData `only` query operator
  * Filter log entries with `$filter`, `$top` and `$skip` queries
  * Delete a Redfish session